In [ ]:
model_path = r"C:\Users\danie\Desktop\RNN3"

In [ ]:
loaded_model = load_model(model_path)

In [ ]:
test_words = pd.read_pickle(r"C:\Users\danie\Documents\GitHub\Masters-Thesis\list_of_words")
test_word_2 = ['is very easy-going person', 'is very dependable person', 'is very fair person', 'is very cooperative person']

#test_words = pd.read_pickle(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/list_of_words")
col_list = test_word_2["feature"].values.tolist()

In [ ]:
X_new = test_word_2 #add word/list of words that we want to predict scores for 
y_pred = loaded_model.predict(X_new)

In [ ]:
Y = pd.DataFrame(y_pred)
Y.reset_index(inplace=True, drop=True)
test_words.reset_index(inplace=True, drop=True)
result = pd.concat([test_words, Y], axis=1)

In [ ]:
pd.set_option('display.max_rows', 500)
result = result.sort_values(by=[1],ascending=False)
result.head(500)